## 합성곱/풀링 계층 구현하기

4차원 배열

In [1]:
import numpy as np

In [3]:
x = np.random.rand(10, 1, 28, 28) # 데이터 수, 채널, 높이, 너비
print(x.shape) 
print(x.shape[0])
print(x.shape[1])

(10, 1, 28, 28)
10
1


im2col로 데이터 전개하기

In [16]:
def im2col(input_data, filter_h, filter_w, stride=1, pad=0):
    N, C, H, W = input_data.shape
    out_h = (H + 2*pad - filter_h) // stride + 1
    out_w = (W + 2*pad - filter_w) // stride + 1

    img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant') ## height, width에만 padding 추가
    col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))

    for y in range(filter_h):
        y_max = y + stride * out_h
        for x in range(filter_w):
            x_max = x + stride * out_w
            col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]
    
    ## N, out_h, out_w, C, h, w -> (N * out_h * out_w, C * h * w)
    col = col.transpose(0, 4, 5, 1, 2 ,3).reshape(N * out_h * out_w, -1)
    return col

In [8]:
input_data = np.random.rand(6, 3, 2, 2)
pad = 1
img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant')

In [13]:
import numpy as np

# 예시 데이터 설정
N, C, H, W = 1, 1, 4, 4  # 1개의 이미지, 1개의 채널, 4x4 크기
filter_h, filter_w, stride, pad = 2, 2, 2, 0  # 2x2 필터, 스트라이드 2, 패딩 0

# 4x4 크기의 이미지 데이터 생성 (값은 행과 열 인덱스의 합으로 설정)
input_data = np.array([[[[i*4 + j for j in range(W)] for i in range(H)]]])
print("원본 이미지 데이터:")
print(input_data)

# 출력 데이터의 높이와 너비 계산
out_h = (H + 2*pad - filter_h)//stride + 1
out_w = (W + 2*pad - filter_w)//stride + 1

# 패딩 적용 (이 경우에는 패딩이 0이므로 원본 데이터 그대로 사용)
img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant')

# 2차원 배열 초기화
col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))

# 이미지 데이터의 평탄화
for y in range(filter_h):
    y_max = y + stride*out_h
    for x in range(filter_w):
        x_max = x + stride*out_w
        col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]

print("\n평탄화된 데이터 (col 배열):")
print(col)

# 차원 재배열 및 최종 2차원 배열 생성
col = col.transpose(0, 4, 5, 1, 2, 3)
print("\ntranspose 배열 (col 배열):")
print(col)

col = col.reshape(N*out_h*out_w, -1)

print("\n최종 2차원 배열 (col 배열):")
print(col)


원본 이미지 데이터:
[[[[ 0  1  2  3]
   [ 4  5  6  7]
   [ 8  9 10 11]
   [12 13 14 15]]]]

평탄화된 데이터 (col 배열):
[[[[[[ 0.  2.]
     [ 8. 10.]]

    [[ 1.  3.]
     [ 9. 11.]]]


   [[[ 4.  6.]
     [12. 14.]]

    [[ 5.  7.]
     [13. 15.]]]]]]

transpose 배열 (col 배열):
[[[[[[ 0.  1.]
     [ 4.  5.]]]


   [[[ 2.  3.]
     [ 6.  7.]]]]



  [[[[ 8.  9.]
     [12. 13.]]]


   [[[10. 11.]
     [14. 15.]]]]]]

최종 2차원 배열 (col 배열):
[[ 0.  1.  4.  5.]
 [ 2.  3.  6.  7.]
 [ 8.  9. 12. 13.]
 [10. 11. 14. 15.]]


합성곱 계층 구현하기

In [17]:
x1 = np.random.rand(1, 3, 7, 7) # 데이터, 채널, 높이, 너비
col1 = im2col(x1, 5, 5, stride=1, pad=0)
print(col1.shape) ## filter_h * filter_w * channel 수만큼의 feature 생성

x2 = np.random.rand(10, 3, 7, 7)
col2 = im2col(x2, 5, 5, stride=1, pad=0)
print(col2.shape)

(9, 75)
(90, 75)


In [18]:
class Convolution:
    def __init__(self, W, b, stride=1, pad=0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad
    
    def forward(self, x):
        FN, C, FH, FW = self.W.shape # 필터 개수, 채널, 높이, 너비
        N, C, H, W = x.shape
        out_h = int((H + 2*self.pad - FH) / self.stride + 1)
        out_w = int((W + 2*self.pad - FW) / self.stride + 1)

        col = im2col(x, FH, FW, self.stride, self.pad) ## (N * H * W, C * FH * FW)
        col_W = self.W.reshape(FN, -1).T # 필터 펼치기 ## (FN, C * FH * FW).T -> (C * FH * FW, FN)
        col_W = self.W.reshape(-1, FN) ## 개선한 코드. 결과적으로 동일하다
        out = np.dot(col, col_W) + self.b

        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)

        return out


In [35]:
## 예시 입력 데이터, 필터 가중치, 편향 설정
x = np.random.rand(1, 1, 4, 4) # 임의의 4x4 크기의 이미지 데이터
weights = np.random.rand(1, 1, 3, 3) # 3x3 크기의 필터
b = np.random.rand(1) # 편향
stride = 1
pad = 0
# Convolution의 forward 함수 대신 독립된 함수로 변환

FN, C, FH, FW = weights.shape # 필터 개수, 채널 수, 필터 높이, 필터 너비 -> 1, 1, 3, 3
N, C, H, W = x.shape # 이미지 수, 채널 수, 높이, 너비 -> 1, 1, 4, 4
out_h = int((H + 2*pad - FH) / stride + 1) # 출력 데이터의 높이 -> 2
out_w = int((W + 2*pad - FW) / stride + 1) # 출력 데이터의 너비 -> 2

# im2col 함수를 사용하여 입력 데이터를 2차원 배열로 변환
col = im2col(x, FH, FW, stride, pad) # (4, 9) = (이미지 수 * 출력 높이 * 출력 너비, 필터 높이 * 필터 너비 * 채널 수)
# 필터를 2차원 배열로 변환
col_weights = weights.reshape(-1, FN)
# col_weights = weights.reshape(FN, -1).T # (1, 9).T = (FN, C * FH * FH).T = (필터 개수, 채널 수 * 필터 높이 * 필터 너비).T

# 컨볼루션 연산 수행: (4, 9) * (9, 1) = (4, 1) = (이미지 수 * 출력 높이 * 출력 너비, 필터 개수)
out = np.dot(col, col_weights) + b
print(out.shape)

# 출력 데이터를 적절한 형태로 재배열
out = out.reshape(N, out_h, out_w, -1) # (이미지 수, 출력 높이, 출력 너비, 필터 개수)

out = out.transpose(0, 3, 1, 2) # (이미지 수, 필터 개수, 출력 높이, 출력 너비)

print("입력 데이터 x:")
print(x)
print("\n필터 가중치 W:")
print(weights)
print("\n편향 b:")
print(b)
print("\n컨볼루션 결과 out:")
print(out)

(4, 1)
입력 데이터 x:
[[[[0.15095727 0.82056136 0.11493924 0.75676101]
   [0.94081566 0.34925049 0.94565707 0.20490969]
   [0.90130254 0.21916467 0.92908738 0.18050672]
   [0.00481105 0.26711582 0.35202711 0.17179169]]]]

필터 가중치 W:
[[[[0.77842886 0.58721771 0.20868072]
   [0.10710841 0.34030623 0.3961517 ]
   [0.51695162 0.39108652 0.97186053]]]]

편향 b:
[0.46235792]

컨볼루션 결과 out:
[[[[3.13453175 2.41899677]
   [2.58539591 2.18616509]]]]
